# High-performance simulations with TFF

This tutorial will describe how to setup high-performance simulations with TFF
in a variety of common scenarios.

NOTE: The mechanisms covered here are not included in the latest release, have
not been tested yet, and the API may evolve. In order to follow this tutorial,
you will need to build a TFF pip package from scratch from the latest sources, and install it in a Jupyter notebook with a Python 3 runtime. The new executor
stack is not compatible with Python 2.

TODO(b/134543154): Populate the content, some of the things to cover here:
- using GPUs in a single-machine setup,
- multi-machine setup on GCP/GKE, with and without TPUs,
- interfacing MapReduce-like backends,
- current limitations and when/how they will be relaxed.

## Before we begin

First, make sure your notebook is connected to a backend that has the relevant
components (including gRPC dependencies for multi-machine scenarios) compiled.

Now, if you are running this notebook in Jupyter, you may need to take an extra
step to work around the
[limitations of Jypter with asyncio](https://github.com/jupyter/notebook/issues/3397#issuecomment-419386811)
by installing the [nest_asyncio](https://github.com/erdewit/nest_asyncio)
package.

In [0]:
!pip install --upgrade nest_asyncio
import nest_asyncio
nest_asyncio.apply()

Now, let's start by loading the MNIST example from the TFF website, and
declaring the Python function that will run a small experiment loop. In order
to use all data and make sure there is enough work for each round, we partition
data from all users into 10 groups and assign one group per simulated client.

In [0]:
!pip install tensorflow_federated
!pip install tf-nightly

In [0]:
import collections
import warnings
import time

import tensorflow as tf
tf.compat.v1.enable_v2_behavior()

import tensorflow_federated as tff
from tensorflow_federated.python.examples import mnist

warnings.simplefilter('ignore')

NUM_EPOCHS = 5
BATCH_SIZE = 100
NUM_CLIENTS = 10

source, _ = tff.simulation.datasets.emnist.load_data()

def client_data(n):
  ids_per_client = int(len(source.client_ids) / NUM_CLIENTS)
  start = ids_per_client * n
  limit = ids_per_client * (n + 1)
  combined_dataset = None
  while start < limit:
    dataset = mnist.keras_dataset_from_emnist(
        source.create_tf_dataset_for_client(source.client_ids[start]))
    start = start + 1
    if combined_dataset is not None:
      combined_dataset = combined_dataset.concatenate(dataset)
    else:
      combined_dataset = dataset
  return combined_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)

train_data = [client_data(n) for n in range(NUM_CLIENTS)]

batch = tf.nest.map_structure(lambda x: x.numpy(), iter(train_data[0]).next())

def model_fn():
  return tff.learning.from_compiled_keras_model(
      mnist.create_simple_keras_model(), batch)

trainer = tff.learning.build_federated_averaging_process(model_fn)

def evaluate(num_rounds=10):
  state = trainer.initialize()
  for _ in range(num_rounds):
    t1 = time.time()
    state, metrics = trainer.next(state, train_data)
    t2 = time.time()
    print('loss {}, round time {}'.format(str(metrics.loss), str(t2 - t1)))

## Single-machine simulations

A simple local multi-threaded executor can be created using a new currently
undocumented framework function `tff.framework.create_local_executor()`, and
made default by calling `tff.framework.set_default_executor()`, as follows.
Note that the elocal executor currently requires the number of clients to be
specified at setup time. We'll relax this restriction in the near future.

In [0]:
tff.framework.set_default_executor(
    tff.framework.create_local_executor(num_clients=NUM_CLIENTS))

evaluate()

loss 3.55805, round time 168.0329692363739
loss 1.20457, round time 162.5828468799591
loss 0.968885, round time 162.3564817905426
loss 0.876231, round time 161.66547751426697
loss 0.822056, round time 165.8971517086029
loss 0.782698, round time 165.80716514587402
loss 0.754549, round time 167.1305811405182
loss 0.735254, round time 162.5927665233612
loss 0.718297, round time 163.8291881084442
loss 0.703123, round time 164.844788312912


Now, for comparison, let's run the same training code using the reference
executor. The reference executor can be automatically installed back by calling
the `tff.framework.set_default_executor()` function without an argument.

In [0]:
tff.framework.set_default_executor()

evaluate()

loss 3.55805, round time 288.4371428489685
loss 1.20457, round time 285.4338390827179
loss 0.968885, round time 279.578590631485
loss 0.876231, round time 281.2864832878113
loss 0.822056, round time 278.5055932998657
loss 0.782698, round time 280.6443078517914
loss 0.754549, round time 280.1886131763458
loss 0.735254, round time 280.347163438797
loss 0.718297, round time 280.31703782081604
loss 0.703123, round time 280.32683181762695


One thing to notice is that whereas the new executor affords a higher resource
utilization (you should see load spread across multiple CPUs), it is fresh off
the oven, and not yet optimized. Consequently, the actual speedup achieved here is about 2x lower than what one might expect simply by comparing CPU
utilization between the two setups. Eliminating the sequential inefficiency in
the eager executor stack is a work in progress. Stay tuned for updates.

## Multi-machine simulations on GCP/GKE, GPUs, TPUs, and beyond...

Coming very soon.